In [ ]:
# Let's first install Cohere's python SDK
# %pip install cohere
# %pip install python-dotenv

In [ ]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

import sys,os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from logger import Logger
from dotenv import load_dotenv

In [ ]:
load_dotenv()
# Paste your API key here. Remember to not share publicly
api_key = os.getenv('COHERE_API_KEY')

# log 
logger = Logger("../logs/jd_entity_extraction.log").get_app_logger()

In [ ]:
# load the dataset
try:
    df = pd.read_csv('../data/Example_data.xlsx')
    logger.info('load data successfully')
except:
    logger.exception('Failed to load dataset')

In [ ]:
# Let's glance at the dataset
df.head(2)

In [ ]:
df.shape

In [ ]:
num_examples = 5
df_sample = df.sample(num_examples,replace=True)

# Split into training and testing sets
sentences_train, sentences_test, labels_train, labels_test = train_test_split(
            list(df_sample[0]), list(df_sample[1]), test_size=0.25, random_state=0)